In [1]:
import numpy as np
from sympy import *
import sympy as sp
init_printing()
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting()

In [43]:
a, b, a3, a4, b3, b4, c, rdw = symbols('a b a_3 a_4 b_3 b_4 c r_dw', type='real')
m, g, I = symbols('m g I', type='real') # g is positive 9.8; I is around center of mass
t3, t4, rc = symbols('theta_3 theta_4 r_c', type='real')
v, w = dynamicsymbols('v omega')
t = symbols('t')

urc, urd, ut, eps = symbols('mu_rc mu_rd mu_tau epsilon')

# COM And Weight Force
Ox, Oy, Oz = symbols('O_x O_y O_z')
O = Matrix([Ox, Oy, Oz])

def rp_matrix(r, p):
    Rx = Matrix([[1, 0, 0], [0, cos(r), -sin(r)], [0, sin(r), cos(r)]])
    Ry = Matrix([[cos(p), 0, sin(p)], [0, 1, 0], [-sin(p), 0, cos(p)]])
    return Ry*Rx

roll, pitch = symbols('theta_roll, theta_pitch')
Fg = rp_matrix(roll, pitch).T * (Matrix([0, 0, -m*g]))

#Standard vectors:
z_vec = Matrix([0,0,0])
ivec = Matrix([1,0,0])
jvec = Matrix([0,1,0])
kvec = Matrix([0,0,1])

In [39]:
# For keeping all variable definitions
def make_var_list():
    strings = []
    names = []

    def vardef(name, xpr):
        s = name + " = " + printing.lambdarepr.lambdarepr(xpr)
        if name in names:
            ind = names.index(name)
            strings[ind] = s
        else:
            strings.append(s)
            names.append(name)
        return s
    
    def add_manual(name, string):
        if name in names:
            ind = names.index(name)
            strings[ind] = string
        else:
            strings.append(string)
            names.append(name)
        return string
    
    def get_all():
        return strings
    
    return vardef, add_manual, get_all
vardef, add_manual, get_all = make_var_list()

def ltx(xpr):
    print("$$" + latex(xpr) + "$$")

# Normal Forces

In [44]:
zo, theta, phi = symbols('z_o theta phi', type='real')
k, n = symbols('k n', type='real')
k1, k2 = k, n*k

#z1 = zo + theta*x1 - phi*y1
z1_xpr = zo + theta*c
z2_xpr = zo + theta*(-c)

#z3 = xo + theta*(x3) - phi*y3
z3_xpr = zo + theta*(a3) - phi*(b3)
z4_xpr = zo + theta*(-a4) - phi*(b4)

#N_i = z_i * k_i
N1 = -k1*z1_xpr
N2 = -k1*z2_xpr
N3 = -k2*z3_xpr
N4 = -k2*z4_xpr

N1 = N1.expand().collect([zo, theta, phi])
N2 = N2.expand().collect([zo, theta, phi])
N3 = N3.expand().collect([zo, theta, phi])
N4 = N4.expand().collect([zo, theta, phi])

In [45]:
Line_1 = N1 + N2 + N3 + N4 # Equals Fw dot k_hat
Line_1 = Line_1.expand().collect([zo, theta, phi])

W_z = symbols('F_N_total')
Eq_1 = Eq(Line_1, W_z)
Eq_1

In [46]:
Line_2 = -N3*b3 - N4*b4 # Equals -(O x Fg) dot i_hat + vdot * m *  O_z**2
Line_2 = Line_2.expand().collect([zo, theta, phi])

Tnet_x = symbols('tau_net_x')
Eq_2 = Eq(Line_2, Tnet_x)
Eq_2

In [47]:
Line_3 = N1*c - N2*c + N3*a3 - N4*a4
Line_3 = Line_3.expand().collect([zo, theta, phi])

Tnet_y = symbols('tau_net_y')
Eq_3 = Eq(Line_3, Tnet_y)
Eq_3

In [51]:
ltx(A)

$$\left[\begin{matrix}- 2 k \left(n + 1\right) & - k n \left(a_{3} - a_{4}\right) & k n \left(b_{3} + b_{4}\right)\\k n \left(b_{3} + b_{4}\right) & k n \left(a_{3} b_{3} - a_{4} b_{4}\right) & - k n \left(b_{3}^{2} + b_{4}^{2}\right)\\- k n \left(a_{3} - a_{4}\right) & - k \left(a_{3}^{2} n + a_{4}^{2} n + 2 c^{2}\right) & k n \left(a_{3} b_{3} - a_{4} b_{4}\right)\end{matrix}\right]$$


In [8]:
A, _ = linear_eq_to_matrix([Line_1, Line_2, Line_3], [zo, theta, phi])
A = factor(A)
A

⎡ -2⋅k⋅(n + 1)        -k⋅n⋅(a₃ - a₄)           k⋅n⋅(b₃ + b₄)   ⎤
⎢                                                              ⎥
⎢                                                 ⎛  2     2⎞  ⎥
⎢k⋅n⋅(b₃ + b₄)      k⋅n⋅(a₃⋅b₃ - a₄⋅b₄)      -k⋅n⋅⎝b₃  + b₄ ⎠  ⎥
⎢                                                              ⎥
⎢                   ⎛  2       2        2⎞                     ⎥
⎣-k⋅n⋅(a₃ - a₄)  -k⋅⎝a₃ ⋅n + a₄ ⋅n + 2⋅c ⎠  k⋅n⋅(a₃⋅b₃ - a₄⋅b₄)⎦

In [9]:
A_prime = A / k
A_prime

⎡  -2⋅n - 2         -n⋅(a₃ - a₄)          n⋅(b₃ + b₄)   ⎤
⎢                                                       ⎥
⎢                                          ⎛  2     2⎞  ⎥
⎢n⋅(b₃ + b₄)     n⋅(a₃⋅b₃ - a₄⋅b₄)      -n⋅⎝b₃  + b₄ ⎠  ⎥
⎢                                                       ⎥
⎢                  2       2        2                   ⎥
⎣-n⋅(a₃ - a₄)  - a₃ ⋅n - a₄ ⋅n - 2⋅c   n⋅(a₃⋅b₃ - a₄⋅b₄)⎦

In [10]:
solns = solve([Eq_1, Eq_2, Eq_3], [zo, theta, phi])

phi_expr = solns[phi]
theta_expr = solns[theta]
zo_expr = solns[zo]

q = Matrix([[zo_expr], [theta_expr], [phi_expr]])
q

⎡              ⎛                   2   ⎛  2     2⎞ ⎛  2       2        2⎞⎞    
⎢<TotalNormal>⋅⎝- n⋅(a₃⋅b₃ - a₄⋅b₄)  + ⎝b₃  + b₄ ⎠⋅⎝a₃ ⋅n + a₄ ⋅n + 2⋅c ⎠⎠ - T
⎢─────────────────────────────────────────────────────────────────────────────
⎢             ⎛ 2          2 ⎛  2     2⎞      2                               
⎢           k⋅⎝n ⋅(a₃ - a₄) ⋅⎝b₃  + b₄ ⎠ - 2⋅n ⋅(a₃ - a₄)⋅(b₃ + b₄)⋅(a₃⋅b₃ - a
⎢                                                                             
⎢                                ⎛          ⎛  2     2⎞                       
⎢              - <TotalNormal>⋅n⋅⎝(a₃ - a₄)⋅⎝b₃  + b₄ ⎠ - (b₃ + b₄)⋅(a₃⋅b₃ - a
⎢          ───────────────────────────────────────────────────────────────────
⎢            ⎛ 2          2 ⎛  2     2⎞      2                                
⎢          k⋅⎝n ⋅(a₃ - a₄) ⋅⎝b₃  + b₄ ⎠ - 2⋅n ⋅(a₃ - a₄)⋅(b₃ + b₄)⋅(a₃⋅b₃ - a₄
⎢                                                                             
⎢                   ⎛                               

In [11]:
B, _ = linear_eq_to_matrix([N1, N2, N3, N4], [zo, theta, phi])
B_prime = B / k
B_prime

⎡-1   -c     0  ⎤
⎢               ⎥
⎢-1    c     0  ⎥
⎢               ⎥
⎢-n  -a₃⋅n  b₃⋅n⎥
⎢               ⎥
⎣-n  a₄⋅n   b₄⋅n⎦

In [42]:
ltx(B_prime)

$$\left[\begin{matrix}-1 & - c & 0\\-1 & c & 0\\- n & - a_{3} n & b_{3} n\\- n & a_{4} n & b_{4} n\end{matrix}\right]$$


In [12]:
add_manual('a_3', "a_3 = a + r_c * cos(theta_3)")
add_manual('a_4', "a_4 = a + r_c * cos(theta_4)")
add_manual('b_3', "b_3 = b - r_c * sin(theta_3)") # MINUS rc*sin(theta_3) because b is a in the negative direction
add_manual('b_4', "b_4 = b - r_c * sin(theta_4)") # MINUS because b is in negative direction

vardef("A_prime", A_prime)
vardef("B_prime", B_prime)

'B_prime = MutableDenseMatrix([[-1, -c, 0], [-1, c, 0], [-n, -a_3*n, b_3*n], [-n, a_4*n, b_4*n]])'

In [13]:
L = Matrix([-Fg.dot(kvec), -O.cross(Fg).dot(ivec) - v.diff(t)*m*Oz**2, -O.cross(Fg).dot(jvec) - m*w.diff(t)*Oz])
vardef("L", L)

'L = MutableDenseMatrix([[g*m*cos(theta_pitch)*cos(theta_roll)], [O_y*g*m*cos(theta_pitch)*cos(theta_roll) - O_z**2*m*Derivative(v(t), t) - O_z*g*m*sin(theta_roll)*cos(theta_pitch)], [-O_x*g*m*cos(theta_pitch)*cos(theta_roll) - O_z*g*m*sin(theta_pitch) - O_z*m*Derivative(omega(t), t)]])'

In [53]:
ltx(L)

$$\left[\begin{matrix}g m \cos{\left (\theta_{pitch} \right )} \cos{\left (\theta_{roll} \right )}\\O_{y} g m \cos{\left (\theta_{pitch} \right )} \cos{\left (\theta_{roll} \right )} - O_{z}^{2} m \frac{d}{d t} v{\left (t \right )} - O_{z} g m \sin{\left (\theta_{roll} \right )} \cos{\left (\theta_{pitch} \right )}\\- O_{x} g m \cos{\left (\theta_{pitch} \right )} \cos{\left (\theta_{roll} \right )} - O_{z} g m \sin{\left (\theta_{pitch} \right )} - O_{z} m \frac{d}{d t} \omega{\left (t \right )}\end{matrix}\right]$$


In [14]:
add_manual("Normals", "N_1, N_2, N_3, N_4 = B_prime.dot(np.linalg.inv(A_prime)).dot(L).T[0]")

'N_1, N_2, N_3, N_4 = B_prime.dot(np.linalg.inv(A_prime)).dot(L).T[0]'

# Castors

In [15]:
#Remake normal force scalars as symbols
N1, N2, N3, N4 = symbols('N_1 N_2 N_3 N_4', type='real')

rc3 = Matrix([rc*cos(t3), rc*sin(t3), 0])
rc4 = Matrix([rc*cos(t4), rc*sin(t4), 0])
v3 = Matrix([b*w, v - a*w, 0])
v4 = Matrix([b*w, v + a*w, 0])
v3_imp = Matrix([b*w.diff(t), v.diff(t) - a*w.diff(t), 0])
v4_imp = Matrix([b*w.diff(t), v.diff(t) + a*w.diff(t), 0])

In [16]:
# Rolling Friction Terms
ang = lambda r_vec, v_vec: (r_vec.dot(v_vec) / (rc * sqrt(v_vec.dot(v_vec)))).simplify()
                            
s_f3_xpr = Piecewise((ang(rc3, v3), v3.dot(v3) > eps), (ang(rc3, v3_imp), v3_imp.dot(v3_imp) > eps), (0, True))       
s_f4_xpr = Piecewise((ang(rc4, v4), v4.dot(v4) > eps), (ang(rc4, v4_imp), v4_imp.dot(v4_imp) > eps), (0, True))
s_f3, s_f4 = symbols('s_f3 s_f4')
Rf3_xpr = Piecewise((-rc3/rc * urc*N3, s_f3 > eps), (rc3/rc * urc*N3, s_f3 < -eps), (z_vec, True))
Rf4_xpr = Piecewise((-rc4/rc * urc*N4, s_f4 > eps), (rc4/rc * urc*N4, s_f4 < -eps), (z_vec, True))
Rf3, Rf4 = symbols('R_f3 R_f4')
Eq(s_f3, s_f3_xpr), Eq(Rf3, Rf3_xpr), Eq(s_f4, s_f4_xpr), Eq(Rf4, Rf4_xpr)

⎛                                                                             
⎜                                                                             
⎜       ⎧ b⋅ω⋅cos(θ₃) + (-a⋅ω + v)⋅sin(θ₃)          2  2             2        
⎜       ⎪ ────────────────────────────────     for b ⋅ω  + (-a⋅ω + v)  > ε    
⎜       ⎪        ____________________                                         
⎜       ⎪       ╱  2  2            2                                          
⎜       ⎪     ╲╱  b ⋅ω  + (a⋅ω - v)                                           
⎜       ⎪                                                                     
⎜s_f3 = ⎨b⋅cos(θ₃)⋅ω̇ + (-a⋅ω̇ + v̇)⋅sin(θ₃)       2   2               2    , 
⎜       ⎪───────────────────────────────────  for b ⋅ω̇  + (-a⋅ω̇ + v̇)  > ε  
⎜       ⎪        _______________________                                      
⎜       ⎪       ╱  2   2              2                                       
⎜       ⎪     ╲╱  b ⋅ω̇  + (a⋅ω̇ - v̇)              

In [17]:
# Friction Torque Terms
cross_ang = lambda rv, vv: (rv.cross(vv)/(rc*sqrt(vv.dot(vv)))).dot(kvec)
s_t3_xpr = Piecewise((cross_ang(rc3, v3), v3.dot(v3) > eps), (cross_ang(rc3, v3_imp), v3_imp.dot(v3_imp) > eps), (0, True))
s_t4_xpr = Piecewise((cross_ang(rc4, v4), v4.dot(v4) > eps), (cross_ang(rc4, v4_imp), v4_imp.dot(v4_imp) > eps), (0, True))
s_t3, s_t4 = symbols('s_t3 s_t4')
Rt3_xpr = Piecewise((-ut*N3 * (rc3.cross(kvec)/rc), s_t3 < -eps), (ut*N3 * (rc3.cross(kvec)/rc), s_t3 > eps), (z_vec, True))
Rt4_xpr = Piecewise((-ut*N4 * (rc4.cross(kvec)/rc), s_t4 < -eps), (ut*N4 * (rc4.cross(kvec)/rc), s_t4 > eps), (z_vec, True))
Rt3, Rt4 = symbols('R_tau_3 R_tau_4')
Eq(s_t3, s_t3_xpr), Eq(Rt3, Rt3_xpr), Eq(s_t4, s_t4_xpr), Eq(Rt4, Rt4_xpr)

⎛                                                                             
⎜                                                                             
⎜      ⎧ -b⋅r_c⋅ω⋅sin(θ₃) + r_c⋅(-a⋅ω + v)⋅cos(θ₃)          2  2             2
⎜      ⎪ ─────────────────────────────────────────     for b ⋅ω  + (-a⋅ω + v) 
⎜      ⎪               _____________________                                  
⎜      ⎪              ╱  2  2             2                                   
⎜      ⎪        r_c⋅╲╱  b ⋅ω  + (-a⋅ω + v)                                    
⎜      ⎪                                                                      
⎜sₜ₃ = ⎨-b⋅r_c⋅sin(θ₃)⋅ω̇ + r_c⋅(-a⋅ω̇ + v̇)⋅cos(θ₃)       2   2              
⎜      ⎪────────────────────────────────────────────  for b ⋅ω̇  + (-a⋅ω̇ + v̇
⎜      ⎪             ________________________                                 
⎜      ⎪            ╱  2   2               2                                  
⎜      ⎪      r_c⋅╲╱  b ⋅ω̇  + (-a⋅ω̇ + v̇)         

In [18]:
vardef("s_f3", s_f3_xpr)
vardef("R_f3", Rf3_xpr)
vardef("s_t3", s_t3_xpr)
vardef("R_tau_3", Rt3_xpr)
add_manual("R_3", "R_3 = R_f3 + R_tau_3")
add_manual('R_3s', "R_3x, R_3y, _ = R_3.T[0]")

vardef("s_f4", s_f4_xpr)
vardef("R_f4", Rf4_xpr)
vardef("s_t4", s_t4_xpr)
vardef("R_tau_4", Rt4_xpr)
add_manual('R_4', "R_4 = R_f4 + R_tau_4")
add_manual('R_4s', "R_4x, R_4y, _ = R_4.T[0]")

'R_4x, R_4y, _ = R_4.T[0]'

# Dynamics

In [19]:
#p1, p2, p3, p4, O = symbols('p_1 p_2 p_3 p_4 O')
#Fw1, Fw2, Fs1, Fs2, Fn1, Fn2, Fn3, Fn4, Fg = symbols('F_w1 F_w2 F_s1 F_s2 F_N1 F_N2 F_N3 F_N4 F_g')

R3x, R3y, R4x, R4y = symbols('R_3x R_3y R_4x R_4y')

p1 = Matrix([-c, 0, 0])
p2 = Matrix([c, 0, 0])
p3 = Matrix([-a, -b, 0])
p4 = Matrix([a, -b, 0]) 

ap1, ap2, bt1, bt2 = symbols('alpha_1 alpha_2 beta_1 beta_2')

Fw1 = Matrix([0, ap1, 0])
Fw2 = Matrix([0, ap2, 0])
Fs1 = Matrix([bt1, 0, 0])
Fs2 = Matrix([bt2, 0, 0])
Fn1 = Matrix([0, 0, N1])
Fn2 = Matrix([0, 0, N2])
Fn3 = Matrix([0, 0, N3])
Fn4 = Matrix([0, 0, N4])
R3 = Matrix([R3x, R3y, 0])
R4 = Matrix([R4x, R4y, 0])

In [20]:
# Sum of Forces
F_LHS = Fw1 + Fw2 + Fs1 + Fs2 + Fn1 + Fn2 + Fn3 + Fn4 + Fg + R3 + R4
F_RHS = Matrix([w.diff(t)*Oy**2*m, v.diff(t)*m, 0])
Eq(F_LHS, F_RHS)

                                                        ⎡   2     ⎤
⎡      R₃ₓ + R₄ₓ + β₁ + β₂ + g⋅m⋅sin(θ_pitch)       ⎤   ⎢O_y ⋅m⋅ω̇⎥
⎢                                                   ⎥   ⎢         ⎥
⎢R_3y + R_4y + α₁ + α₂ - g⋅m⋅sin(θᵣₒₗₗ)⋅cos(θ_pitch)⎥ = ⎢  m⋅v̇   ⎥
⎢                                                   ⎥   ⎢         ⎥
⎣  N₁ + N₂ + N₃ + N₄ - g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ)  ⎦   ⎣    0    ⎦

In [21]:
M_LHS = p1.cross(Fw1) + p2.cross(Fw2) + p1.cross(Fs1) + p2.cross(Fs2) + O.cross(Fg) + p1.cross(Fn1) + p2.cross(Fn2) + p3.cross(Fn3) + p4.cross(Fn4) + p3.cross(R3) + p4.cross(R4)
M_RHS = Matrix([v.diff(t)*m*Oz**2, 0, I*w.diff(t)])
Eq(M_LHS, M_RHS)

                                                                              
⎡           -N₃⋅b - N₄⋅b - O_y⋅g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ) + O_z⋅g⋅m⋅sin(θᵣₒₗₗ
⎢                                                                             
⎢          N₁⋅c - N₂⋅c + N₃⋅a - N₄⋅a + Oₓ⋅g⋅m⋅cos(θ_pitch)⋅cos(θᵣₒₗₗ) + O_z⋅g⋅
⎢                                                                             
⎣-Oₓ⋅g⋅m⋅sin(θᵣₒₗₗ)⋅cos(θ_pitch) - O_y⋅g⋅m⋅sin(θ_pitch) + R₃ₓ⋅b - R_3y⋅a + R₄ₓ

                             ⎡   2     ⎤
)⋅cos(θ_pitch)           ⎤   ⎢O_z ⋅m⋅v̇⎥
                         ⎥   ⎢         ⎥
m⋅sin(θ_pitch)           ⎥ = ⎢    0    ⎥
                         ⎥   ⎢         ⎥
⋅b + R_4y⋅a - α₁⋅c + α₂⋅c⎦   ⎣  I⋅ω̇   ⎦

In [22]:
eq1 = Eq(F_LHS[1], F_RHS[1]) #Pick equation of Fy
eq2 = Eq(M_LHS[2], M_RHS[2]) # Pick equation of Mz
eq1, eq2

In [23]:
ap_dict = solve([eq1, eq2], [ap1, ap2])
ap1_xpr = ap_dict[ap1]
ap2_xpr = ap_dict[ap2]

In [24]:
vardef("alpha_1", ap1_xpr)
vardef("alpha_2", ap2_xpr)

'alpha_2 = (I*Derivative(omega(t), t) + O_x*g*m*sin(theta_roll)*cos(theta_pitch) + O_y*g*m*sin(theta_pitch) - R_3x*b + R_3y*a - R_4x*b - R_4y*a - c*(R_3y + R_4y - g*m*sin(theta_roll)*cos(theta_pitch) - m*Derivative(v(t), t)))/(2*c)'

# Drive Wheels

In [25]:
v1 = v - w*c
v2 = v + w*c
Rf_1_xpr = Piecewise((-urd * N1, v1 > eps), (urd*N1, v1 < -eps), (0, True))
Rf_2_xpr = Piecewise((-urd * N2, v2 > eps), (urd*N1, v2 < -eps), (0, True))

Torque_1 = (ap1_xpr - Rf_1_xpr) * rdw
Torque_2 = (ap2_xpr - Rf_2_xpr) * rdw

In [26]:
vardef("torque_1", Torque_1)
vardef("torque_2", Torque_2)

'torque_2 = r_dw*(-((-N_2*mu_rd) if (c*omega(t) + v(t) > epsilon) else (((N_1*mu_rd) if (c*omega(t) + v(t) < -epsilon) else (((0) if (True) else None))))) + (I*Derivative(omega(t), t) + O_x*g*m*sin(theta_roll)*cos(theta_pitch) + O_y*g*m*sin(theta_pitch) - R_3x*b + R_3y*a - R_4x*b - R_4y*a - c*(R_3y + R_4y - g*m*sin(theta_roll)*cos(theta_pitch) - m*Derivative(v(t), t)))/(2*c))'

# Translating Sympy to Code

In [39]:
def translate_str(s):
    s1 = s.replace("MutableDenseMatrix", 'array')
    s2 = s1.replace("ImmutableDenseMatrix", 'array')
    s3 = s2.replace("Derivative(omega(t), t)", "omega_dot")
    s4 = s3.replace("Derivative(v(t), t)", "v_dot")
    s5 = s4.replace("(t)", "")
    s6 = s5.replace("Abs", "abs")
    return s6

In [40]:
filestr = ""
for s in get_all():
    filestr += translate_str(s) + "\n"

print(filestr)


a_3 = a + r_c * cos(theta_3)
a_4 = a + r_c * cos(theta_4)
b_3 = b - r_c * sin(theta_3)
b_4 = b - r_c * sin(theta_4)
A_prime = array([[-2*n - 2, -n*(a_3 - a_4), n*(b_3 + b_4)], [n*(b_3 + b_4), n*(a_3*b_3 - a_4*b_4), -n*(b_3**2 + b_4**2)], [-n*(a_3 - a_4), -a_3**2*n - a_4**2*n - 2*c**2, n*(a_3*b_3 - a_4*b_4)]])
B_prime = array([[-1, -c, 0], [-1, c, 0], [-n, -a_3*n, b_3*n], [-n, a_4*n, b_4*n]])
L = array([[g*m*cos(theta_pitch)*cos(theta_roll)], [O_y*g*m*cos(theta_pitch)*cos(theta_roll) - O_z**2*m*v_dot - O_z*g*m*sin(theta_roll)*cos(theta_pitch)], [-O_x*g*m*cos(theta_pitch)*cos(theta_roll) - O_z*g*m*sin(theta_pitch) - O_z*m*omega_dot]])
N_1, N_2, N_3, N_4 = B_prime.dot(np.linalg.inv(A_prime)).dot(L).T[0]
s_f3 = (((b*omega*cos(theta_3) + (-a*omega + v)*sin(theta_3))/sqrt(b**2*omega**2 + (a*omega - v)**2)) if (b**2*omega**2 + (-a*omega + v)**2 > epsilon) else ((((b*cos(theta_3)*omega_dot + (-a*omega_dot + v_dot)*sin(theta_3))/sqrt(b**2*omega_dot**2 + (a*omega_dot - v_dot)**2)) if (b**2*omega

In [34]:
funcstr = """import numpy as np
from numpy import cos, sin, sqrt, array

a = 0.2
b = 0.4
c = 0.2
r_c = 0.05
r_dw = 0.1
m = 230
n = 100
mu_rc = 0.01
mu_rd = 0.08
mu_tau = 0.02

O_x, O_y, O_z = 0, -0.1, 0.25

epsilon = 0.03

def calc(roll, pitch, theta_3, theta_4, v, v_dot, omega, omega_dot, a=a, b=b, c=c, r_c=r_c, r_dw=r_dw, m=m, n=n, epsilon=epsilon, mu_rc=mu_rc, mu_rd=mu_rd, mu_tau=mu_tau,
         O_x=O_x, o_y = O_y, O_z = O_z):
    theta_pitch = pitch
    theta_roll = roll
    I = m*(4*c**2 + b**2)/12
    g = 9.8
"""
for s in get_all():
    funcstr += "    " + translate_str(s) + '\n'
    
funcstr += """
    d = {'N1': N_1, 'N2': N_2, 'N3': N_3, 'N4': N_4, 'sf3': s_f3, 'sf4': s_f4, 'st3': s_t3, 'st4': s_t4, 'R3x': R_3x, 'R3y': R_3y,
         'R4x': R_4x, 'R4y': R_4y, 'a1': alpha_1, 'a2': alpha_2, 'torque_1': torque_1, 'torque_2': torque_2}
    return d"""

if (False) :
    print(funcstr)
else:
    f = open('WheelchairFunc.py', 'w+')
    f.write(funcstr)
    f.close()